## Maximal Marginal Relevance

In [2]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.retrievers import BM25Retriever
from langchain_classic.retrievers import EnsembleRetriever
from langchain_core.documents import Document
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
# chain creation
from langchain_classic.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.output_parsers import StrOutputParser 
from langchain.chat_models import init_chat_model
from langchain_groq import ChatGroq
from dotenv import load_dotenv

In [ ]:
import os
load_dotenv()
groq_llm = ChatGroq(model="llama-3.1-8b-instant",api_key=os.getenv("GROQ_API"),temperature=0.2)


In [4]:
loader = TextLoader("حباب سکه چیست.txt",encoding="utf-8")
loader = loader.load()


In [5]:
splitter = RecursiveCharacterTextSplitter(
    separators=[" "],
    chunk_size=600,
    chunk_overlap=200 # specially make it more to increase relevancy 
)
docs = splitter.split_documents(loader)
for d in docs[:-3]:
    print("*"*50)
    print(d)

**************************************************
page_content='یکی از مفاهیم مهم برای خرید و فروش در بازار سکه، آشنایی با حباب سکه است. احتمالا اصطلاح حباب زیاد به گوش‌تان خورده است که برای انواع بازارها به کار برده می‌شود. این اصطلاح در واقع به تغییرات قیمت در عرضه و تقاضا و حرکت آن به سمت مثبت یا منفی اشاره دارد. اما حباب سکه پیچیده‌تر از این‌هاست! اگر به سرمایه‌ گذاری در بازار سکه علاقه دارید باید به طور کامل با حباب این بازار آشنایی داشته باشید. به این منظور، لازم است که نحوه محاسبه حباب سکه را بدانید و عوامل موثر بر آن را تشخیص دهید. در این مطلب از ره‌آورد، به طور کامل در مورد حباب سکه مطالعه خواهید کرد.

حباب سکه چیست؟
در ابتدا بیایید ماهیت حباب سکه' metadata={'source': 'حباب سکه چیست.txt'}
**************************************************
page_content='لازم است که نحوه محاسبه حباب سکه را بدانید و عوامل موثر بر آن را تشخیص دهید. در این مطلب از ره‌آورد، به طور کامل در مورد حباب سکه مطالعه خواهید کرد.

حباب سکه چیست؟
در ابتدا بیایید ماهیت حباب سکه را به خوبی بررسی کنیم. بدون شک،

In [6]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

vectorstore = FAISS.from_documents(docs, embedding_model)

retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k":3})

In [ ]:
system_prompt = """
تو یک دستیار فارسی‌زبان مفید هستی.
از اطلاعات داده‌شده برای پاسخ دقیق به سوال استفاده کن.
پاسخ باید روان، طبیعی، و مناسب خواندن توسط انسان باشد.
پاراگراف‌بندی منطقی انجام بده، اما از ایجاد خطوط خالی یا شکست‌های غیرضروری خودداری کن.
فقط از حروف و نشانه‌های زبان فارسی استفاده کن.
از استفاده از هر زبان یا کاراکتر غیر فارسی (مانند چینی، انگلیسی یا عربی) خودداری کن.


اطلاعات:
{context}
سوال کاربر : {question}
"""
template = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                ("human", "{question}"),
            ]
        )
template

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nتو یک دستیار فارسی\u200cزبان مفید هستی.\nاز اطلاعات داده\u200cشده برای پاسخ دقیق به سوال استفاده کن.\nپاسخ باید روان، طبیعی، و مناسب خواندن توسط انسان باشد.\nپاراگراف\u200cبندی منطقی انجام بده، اما از ایجاد خطوط خالی یا شکست\u200cهای غیرضروری خودداری کن.\n\nاطلاعات:\n{context}\nسوال کاربر : {question}\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])

In [18]:
chain = template | groq_llm | StrOutputParser()
chain

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='تو یک دستیار مفید هستی که از اطلاعات دریافتی باید به سوال کاربر پاسخ دهی \nاطلاعات : {context}\nسوال کاربر : {question}\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])
| ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000020982528D70>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000020982529A90

In [34]:
response = chain.invoke({"context":docs,"question":"یک مثال از حباب سکه بگو "})
print(response)

بسیار خوب، یک مثال از حباب سکه به این صورت می‌تواند باشد:

فرض کنید قیمت روز یک سکه 10 میلیون تومان است و ارزش ذاتی آن 9 میلیون تومان است. در این صورت، حباب سکه 10 میلیون تومان - 9 میلیون تومان = 1 میلیون تومان است.

این به معنای آن است که سکه در بازار با قیمتی بالاتر از ارزش ذاتی خود معامله می‌شود و حباب آن 1 میلیون تومان است.


In [36]:
response = chain.invoke({"context":docs,"question":"یک مثال از حباب سکه بگو "})
print(response)

مثال:

فرض کنید قیمت واقعی یک سکه در بازار 10 میلیون تومان باشد. همچنین قیمت طلا در بازار جهانی 30 هزار تومان در گرم است. همچنین قیمت دلار در بازار 10 هزار تومان است. در این صورت، اگر سکه بهار آزادی را در نظر بگیریم، با توجه به این که وزن آن 8.1333 گرم است، در فرمول محاسبه حباب سکه، ارزش ذاتی آن به این صورت محاسبه می شود:

ارزش ذاتی = (8.1333 * 30,000 * 0.9 * 31.1 * 10,000) / (8.1333 * 31.1 * 10,000 + 5,000) = 25,000,000

حالا اگر قیمت سکه در بازار 10,100,000 تومان باشد، حباب سکه به این صورت محاسبه می شود:

حباب = قیمت سکه - ارزش ذاتی = 10,100,000 - 25,000,000 = -15,000,000

با این فرض که قیمت سکه در بازار 10,100,000 تومان باشد، حباب سکه منفی است و نشان دهنده این است که قیمت سکه کمتر از ارزش ذاتی آن است.

اما اگر قیمت سکه در بازار 10,500,000 تومان باشد، حباب سکه به این صورت محاسبه می شود:

حباب = قیمت سکه - ارزش ذاتی = 10,500,000 - 25,000,000 = -14,500,000

در این مثال، حباب سکه مثبت است و نشان دهنده این است که قیمت سکه بیشتر از ارزش ذاتی آن است.
